In [29]:
import pandas as pd
df = pd.read_csv('assets.csv')
df.head()
# drop the rows with missing '物体种类'
df = df.dropna(subset=['物体种类'])
len(df)
import json
with open(r'D:\code\LEGENT\LEGENT\legent\scene_generation\data\addressables.json','r',encoding='utf-8') as f:
    data = json.load(f)
data = set(item['name'] for item in data['prefabs'])

import re
def camel_to_snake(name):
    # 在大写字母前面插入下划线，并将结果转换为小写
    s = name
    s = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)  # 在小写字母和大写字母之间插入下划线
    s = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s)  # 在小写字母/数字和大写字母之间插入下划线
    s = s.lower()
    s = '_'.join(s.strip().split())
    s = re.sub(r'_+', '_', s)
    return s
# print(camel_to_snake("smallTable"))  # 输出：small_table
# print(camel_to_snake("SmallTable"))  # 输出：small_table
# print(camel_to_snake("ThisIsATest"))  # 输出：this_is_a_test
# print(camel_to_snake("HTTPResponseCode"))  # 输出：http_response_code
# print(camel_to_snake(" wall light "))  # 输出：wall_light
# print(camel_to_snake('Cup'))
remove_objects = set(["LowPolyInterior2_Sink1_C1",
    "LowPolyInterior2_Sink1_C2",
    "LowPolyInterior2_Sink1_C3",
    "LowPolyInterior2_Sink2_C1",
    "LowPolyInterior2_Sink2_C2",
    "LowPolyInterior2_Sink2_C3",
    'LowPolyInterior2_Carpet_02',
    'LowPolyInterior2_Carpet_04',
    'LowPolyInterior2_Carpet_06'])


def convert_name(path):
    group = path.split('/')[1]
    name = path.split('/')[-1].split('.')[0]
    return group + '_' + name
res = {}
for i, row in df.iterrows():
    name = convert_name(row['文件'])
    cat = row['物体种类']
    if name not in data:
        pass
    elif name in remove_objects:
        pass
    else:
        res[name] = camel_to_snake(cat)
with open('new_data/object_name_to_type.json','w') as f:
    json.dump(res,f,ensure_ascii=False, indent=2)
object_dict = {}
for k,v in res.items():
    if v not in object_dict:
        object_dict[v] = []
    object_dict[v].append(k)
with open('new_data/object_dict.json','w') as f:
    json.dump(object_dict,f,ensure_ascii=False, indent=2)
categories = set(res.values())
with open('new_data/categories.json','w') as f:
    json.dump(list(categories),f,ensure_ascii=False, indent=2)
print(', '.join(sorted(categories)))

air_conditioner, apple, avocado, bacon, banana, bar, bar_chair, bath_carpet, bathtub, bed, bed_table, beet, bench, book, bottle, bowl, box, bread, broccoli, bun, burger, butter, cabbage, cabinet, cake, candle, car, carpet, carrot, ceiling_fan, ceiling_light, certificate, cezve, chair, champagne, cheese, cherry, chicken, chili, chocolate, christmas_lollipop, christmas_tree, clock, clothes, coffee_cup, coffee_machine, cola, cookie, corn, countertop, cucumber, cup, cupcake, cutting_board, dishwasher, donut, drainer, dress, dresser, dumbbell, egg, eggplant, fireplace, fish, floor_lamp, football, fork, frame, french_fries, game_console, gamepad, garlic, gift_box, globe, grape, hob, hot_dog, hotdog, jar, ketchup, kettle, keyboard, kitchen_chair, kitchen_desk, kitchen_hanger, kitchen_hood, kitchen_table, kiwi, knife, kubik_rubik, lamp, laptop, lemon, lollipop, microwave, mirror, mouse_mat, mushroom, mustard, office_table, officechair, onion, orange, oven, painting, pan, pancakes, pc, pc_chair

In [30]:
path = r'D:\code\LEGENT\LEGENT\legent\scene_generation\data_annotation\new_data\annotation.jsonl'
import json
with open(path,'r',encoding='utf-8') as f:
    data = [json.loads(line) for line in f]
# print(data[0])
def filter_object_cat(cat):
    if cat.startswith('ceiling'):
        return False
    # elif cat.startswith('floor'):
    #     return False
    elif cat.startswith('wall'):
        return False
    return True
data = {item['object']: json.loads(item['annotation']) for item in data if filter_object_cat(item['object'])}
import pandas as pd
df = pd.DataFrame(data).T
df.sort_index(inplace=True)
# set the index name
df.index.name = 'Object'

df['inBathrooms'] *=2
df['inBedrooms'] *=2
df['inKitchens'] *=2
df['inLivingRooms'] *=2
# set onFloor col to bool
df['onFloor'] = df['onFloor'].astype(bool)

df['onEdge'] = True
df['inMiddle'] = True
df['inCorner'] = True
df['multiplePerRoom'] = True
# handcraft the placement annotations
FLOOR = 'onFloor'
KITCHEN = 'inKitchens'
LIVING_ROOM = 'inLivingRooms'
BEDROOM = 'inBedrooms'
BATHROOM = 'inBathrooms'
MULTI = 'multiplePerRoom'
MIDDLE = 'inMiddle'

not_floor_objects = '''
game_console
shelf
dumbbell
tv
slippers
shoe
toy
box
plumbing
clothes
pc_screen
lamp
towel_rail
globe
air_conditioner
countertop
wok
toaster
mouse_mat
printer
coffee_machine
'''
for obj in not_floor_objects.strip().split('\n'):
    df.loc[obj,FLOOR] = False

no_middle_objects = '''
sink
plant
sofa
small_table
bed_table
tv_table
pc_table
trash_bin
toilet
washing_machine
bench
refrigerator
shower_enclosure
'''
for obj in no_middle_objects.strip().split('\n'):
    df.loc[obj,MIDDLE] = False

no_multi_objects = '''
tv
trash_bin
bar
toilet_plunger
toilet
'''
for obj in no_multi_objects.strip().split('\n'):
    df.loc[obj,MULTI] = False

df.to_csv('../data/placement_annotations.csv')
floor_df = df[df['onFloor'] == 1]
floor_objects = floor_df.index.values.tolist()
with open('../data/floor_objects.json','w',encoding='utf-8') as f:
    json.dump(floor_objects,f,indent=4,ensure_ascii=False)
not_floor_df = df[df['onFloor'] == 0]
not_floor_objects = not_floor_df.index.values.tolist()
print(', '.join(not_floor_objects))

air_conditioner, apple, avocado, bacon, banana, beet, book, bottle, bowl, box, bread, broccoli, bun, burger, butter, cabbage, cake, candle, carrot, certificate, cezve, champagne, cheese, cherry, chicken, chili, chocolate, christmas_lollipop, clock, clothes, coffee_cup, coffee_machine, cola, cookie, corn, countertop, cucumber, cup, cupcake, cutting_board, donut, drainer, dress, dumbbell, egg, eggplant, fish, fork, frame, french_fries, game_console, gamepad, garlic, gift_box, globe, grape, hot_dog, jar, ketchup, kettle, keyboard, kitchen_hanger, kiwi, knife, kubik_rubik, lamp, laptop, lemon, lollipop, mirror, mouse_mat, mushroom, mustard, onion, orange, painting, pan, pancakes, pc, pc_mouse, pc_screen, pear, pen_holder, pendulum, pepper, photo_frame, pie, pillow, pineapple, pizza, plate, plumbing, pot_cover, potato, printer, pumpkin, sandwich, santizer, sausage, seasoning_jar, shelf, shoe, shorts, shower_gel, shrimp, slippers, soda, soy_sauce_bottle, soy_sauce_bowl, spatula, spoon, srub_

In [31]:
path = r'D:\code\LEGENT\LEGENT\legent\scene_generation\data_annotation\new_data\annotation_receptacle.jsonl'
data = []
with open(path,'r',encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
data = {item['object']:item['annotation'] for item in data}
def is_chinese_char(char):
    return '\u4e00' <= char <= '\u9fa5'
def convert_to_objects(text):
    # 第一个字符是否为汉字
    text = text.split('可以摆放的物体：')[-1].strip()
    if is_chinese_char(text[0]):
        return {}
    return {k:2  for k in text.split(', ') if k in categories}
data = {k:convert_to_objects(v) for k,v in data.items()}
data['officechair'] = data['office_table']
data['bar_chair'] = data['bar']
data['pc_chair'] = data['pc_table']
data['kitchen_chair'] = data['kitchen_table']
data['chair'] = data['table']


data
with open('../data/receptacle.json','w') as f:
    json.dump(data,f,ensure_ascii=False, indent=2)